# Example: Tool Usage of Responses API

This example demonstrates how to use the Responses API to interact with a custom tool. The workflow includes defining a tool schema, handling function calls, and providing outputs back to the model. The example showcases a weather-fetching tool that responds to user queries about the weather in a specific location.

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import requests
import json

# Get OPENAI_API_KEY from .env file
load_dotenv()
client = OpenAI()

### Step 1: Try Custom Function

In this step, we define and use a custom function `get_weather` to fetch the current temperature for a given set of coordinates. Later, we will explore using OpenAI's built-in capabilities for similar tasks.

In [3]:
def get_weather(latitude, longitude):
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]["temperature_2m"]

In [4]:
# Define the tool's schema
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"},
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False,
        },
        "strict": True,
    }
]

In [5]:
input_messages = [
    {"role": "user", "content": "What's the weather like in Tokyo today?"}
]

In [6]:
response = client.responses.create(
    model="gpt-4o",
    input=input_messages,
    tools=tools,
)

In [7]:
response.output

[ResponseFunctionToolCall(arguments='{"latitude":35.682839,"longitude":139.759455}', call_id='call_KqM2XlFWwa9VKR6A35JbXK1m', name='get_weather', type='function_call', id='fc_67fcae117d6881918df7e4b719a9abd10a45c2aed1083c1b', status='completed')]

In [8]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = get_weather(args["latitude"], args["longitude"])

In [9]:
result

20.5

In [10]:
input_messages.append(tool_call)  # append model's function call message
input_messages.append(
    {  # append result message
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result),
    }
)

response_2 = client.responses.create(
    model="gpt-4o",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

The weather in Tokyo today is 20.5°C.


### Step 2: Built-in Functions

In this step, we explore using OpenAI's built-in functions to handle tasks such as fetching weather data. This approach leverages the model's capabilities to directly interpret and execute function calls, simplifying the workflow and reducing the need for custom implementations.

In [11]:
response = client.responses.create(
    model="gpt-4o",
    tools=[{"type": "web_search_preview"}],
    input="What's the weather like in Tokyo today?",
)

print(response.output_text)

As of 6:41 AM JST on Monday, April 14, 2025, the weather in Tokyo is mostly sunny with a temperature of 70°F (21°C).

## Weather for Tokyo, Japan:
Current Conditions: Mostly sunny, 70°F (21°C)

Hourly Forecast:
* 4:00 PM: 68°F (20°C), Sunny
* 5:00 PM: 67°F (19°C), Sunny
* 6:00 PM: 64°F (18°C), Sunny
* 7:00 PM: 63°F (17°C), Intermittent clouds
* 8:00 PM: 61°F (16°C), Intermittent clouds
* 9:00 PM: 60°F (16°C), Intermittent clouds
* 10:00 PM: 60°F (16°C), Intermittent clouds
* 11:00 PM: 60°F (16°C), Intermittent clouds
* 12:00 AM: 60°F (15°C), Intermittent clouds
* 1:00 AM: 59°F (15°C), Intermittent clouds
* 2:00 AM: 59°F (15°C), Intermittent clouds
* 3:00 AM: 58°F (14°C), Intermittent clouds

Severe Weather Alerts:
* Shinjuku-ku: Thunderstorm Advisory in effect until 6:00 PM JST.  Source: Japan Meteorological Agency, , Start time: Monday, April 14, 12:00 PM, End time: Monday, April 14, 06:00 PM


Please be aware that a Thunderstorm Advisory is in effect for Shinjuku-ku until 6:00 PM JST

In [12]:
response

Response(id='resp_67fcae27f480819184c8ce682553e906000f6f144f73dc15', created_at=1744612903.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ResponseFunctionWebSearch(id='ws_67fcae28b1cc819187d18242ea810db4000f6f144f73dc15', status='completed', type='web_search_call'), ResponseOutputMessage(id='msg_67fcae2a819c8191b2c086cd9ebcc632000f6f144f73dc15', content=[ResponseOutputText(annotations=[AnnotationURLCitation(end_index=1420, start_index=1321, title='April weather - Spring 2025 - Tokyo, Japan', type='url_citation', url='https://www.weather-atlas.com/en/japan/tokyo-weather-april?utm_source=openai')], text="As of 6:41 AM JST on Monday, April 14, 2025, the weather in Tokyo is mostly sunny with a temperature of 70°F (21°C).\n\n## Weather for Tokyo, Japan:\nCurrent Conditions: Mostly sunny, 70°F (21°C)\n\nHourly Forecast:\n* 4:00\u202fPM: 68°F (20°C), Sunny\n* 5:00\u202fPM: 67°F (19°C), Sunny\n* 6:00\u202fPM: 64°F (